In [42]:
# Reading and data and display sample

import pandas as pd
import os

path = os.path.join(os.getcwd(),'data/nikhil_onlytextpytorch.json')
print(path)

train = pd.read_json(path, orient='index')
display(train)

/Users/nik/Documents/Codes/random_stuff/TSAI_END2_Phase1/data/nikhil_onlytextpytorch.json


,answer,question,context,source
"What values are specified to replaceNaN, positive infinity, and negative infinity values in input?","bynan,posinf, andneginf","What values are specified to replaceNaN, posit...","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
"By default,NaN is replaced with what value?",zero,"By default,NaN is replaced with what value?","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
What values are used to replace negative infinity values in input?,"bynan,posinf, andneginf",What values are used to replace negative infin...,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
What is the default value for negative infinity?,the least finite value,What is the default value for negative infinity?,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
What is the value to replace positive infinity values with?,posinf,What is the value to replace positive infinity...,"nan(Number,optional) – the value to replaceNaN...",https://pytorch.org/docs/stable/generated/torc...
...,...,...,...,...
What is the tensor of shape where*is zero or more batch dimensions?,A(Tensor),What is the tensor of shape where*is zero or m...,torch.linalg.inv()computes the inverse of a sq...,https://pytorch.org/docs/stable/generated/torc...
What is the tolerance value to determine when is a singular value zero?,rcond,What is the tolerance value to determine when ...,torch.linalg.lstsq()computesA.pinv() @Bwith a\...,https://pytorch.org/docs/stable/generated/torc...
What is the default value of atorch.Tensor?,1e-15,What is the default value of atorch.Tensor?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...
What computes the inverse of a square matrix?,torch.linalg.inv(),What computes the inverse of a square matrix?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...


In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f'device is {device}')

device is cpu


In [4]:
data = 'data/nikhil_onlytextpytorch.json'
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading: 100%|██████████| 232k/232k [00:03<00:00, 65.1kB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 16.7kB/s]
Downloading: 100%|██████████| 466k/466k [00:01<00:00, 389kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 258kB/s]


In [23]:
def read_qna_file(input_file):
    """Read a  json file into a list and return data."""
    with open(input_file, "r", encoding='utf-8') as reader:
        input_data = json.load(reader)["data"]
        return input_data

def tokenize(input_text, tokenizer, device):
    """Tokenize the input text using tokenizer"""
    tokens = tokenizer(input_text, return_tensors="pt").to(device)


# Add CLS and SEP tokens to the device
def tokenize(input_text, tokenizer, device):
    """Add tokens requrired for BERT to the training data"""
    ['CLS'] ['SEP']
# Convert string into tokens and back




1016

In [15]:
print(len(data.keys()))

1003


In [10]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(data, test_size=.2)


KeyError: 37

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import torch

class qna_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:

# Adding tokens


doc_stride=''
max_query_length = 100
max_seq_length=100
cls_token='[CLS]'
sep_token='[SEP]'
pad_token=0
sequence_a_segment_id=0
sequence_b_segment_id=1,
cls_token_segment_id=0
pad_token_segment_id=0,
mask_padding_with_zero=True


unique_id = 1000000000

features = []

query_tokens = tokenizer.tokenize(example.question_text)

if len(query_tokens) > max_query_length:
    query_tokens = query_tokens[0:max_query_length]

    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    for (i, token) in enumerate(example.doc_tokens):
        orig_to_tok_index.append(len(all_doc_tokens))
        sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
            tok_to_orig_index.append(i)
            all_doc_tokens.append(sub_token)

    tok_start_position = None
    tok_end_position = None
    if is_training and example.is_impossible:
        tok_start_position = -1
        tok_end_position = -1
    if is_training and not example.is_impossible:
        tok_start_position = orig_to_tok_index[example.start_position]
        if example.end_position < len(example.doc_tokens) - 1:
            tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
        else:
            tok_end_position = len(all_doc_tokens) - 1
        (tok_start_position, tok_end_position) = _improve_answer_span(
            all_doc_tokens, tok_start_position, tok_end_position, tokenizer,
            example.orig_answer_text)

    # The -3 accounts for [CLS], [SEP] and [SEP]
    max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_tokens_for_doc:
            length = max_tokens_for_doc
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)

    for (doc_span_index, doc_span) in enumerate(doc_spans):
        tokens = []
        token_to_orig_map = {}
        token_is_max_context = {}
        segment_ids = []

        # p_mask: mask with 1 for token than cannot be in the answer (0 for token which can be in an answer)
        # Original TF implem also keep the classification token (set to 0) (not sure why...)
        p_mask = []

        # CLS token at the beginning
        if not cls_token_at_end:
            tokens.append(cls_token)
            segment_ids.append(cls_token_segment_id)
            p_mask.append(0)
            cls_index = 0

        # Query
        for token in query_tokens:
            tokens.append(token)
            segment_ids.append(sequence_a_segment_id)
            p_mask.append(1)

        # SEP token
        tokens.append(sep_token)
        segment_ids.append(sequence_a_segment_id)
        p_mask.append(1)

        # Paragraph
        for i in range(doc_span.length):
            split_token_index = doc_span.start + i
            token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

            is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                    split_token_index)
            token_is_max_context[len(tokens)] = is_max_context
            tokens.append(all_doc_tokens[split_token_index])
            segment_ids.append(sequence_b_segment_id)
            p_mask.append(0)
        paragraph_len = doc_span.length

        # SEP token
        tokens.append(sep_token)
        segment_ids.append(sequence_b_segment_id)
        p_mask.append(1)

        # CLS token at the end
        if cls_token_at_end:
            tokens.append(cls_token)
            segment_ids.append(cls_token_segment_id)
            p_mask.append(0)
            cls_index = len(tokens) - 1  # Index of classification token

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(pad_token)
            input_mask.append(0 if mask_padding_with_zero else 1)
            segment_ids.append(pad_token_segment_id)
            p_mask.append(1)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        span_is_impossible = example.is_impossible
        start_position = None
        end_position = None
        if is_training and not span_is_impossible:
            # For training, if our document chunk does not contain an annotation
            # we throw it out, since there is nothing to predict.
            doc_start = doc_span.start
            doc_end = doc_span.start + doc_span.length - 1
            out_of_span = False
            if not (tok_start_position >= doc_start and
                    tok_end_position <= doc_end):
                out_of_span = True
            if out_of_span:
                start_position = 0
                end_position = 0
                span_is_impossible = True
            else:
                doc_offset = len(query_tokens) + 2
                start_position = tok_start_position - doc_start + doc_offset
                end_position = tok_end_position - doc_start + doc_offset

        if is_training and span_is_impossible:
            start_position = cls_index
            end_position = cls_index

        if example_index < 20:
            logger.info("*** Example ***")
            logger.info("unique_id: %s" % (unique_id))
            logger.info("example_index: %s" % (example_index))
            logger.info("doc_span_index: %s" % (doc_span_index))
            logger.info("tokens: %s" % " ".join(tokens))
            logger.info("token_to_orig_map: %s" % " ".join([
                "%d:%d" % (x, y) for (x, y) in token_to_orig_map.items()]))
            logger.info("token_is_max_context: %s" % " ".join([
                "%d:%s" % (x, y) for (x, y) in token_is_max_context.items()
            ]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info(
                "input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            if is_training and span_is_impossible:
                logger.info("impossible example")
            if is_training and not span_is_impossible:
                answer_text = " ".join(tokens[start_position:(end_position + 1)])
                logger.info("start_position: %d" % (start_position))
                logger.info("end_position: %d" % (end_position))
                logger.info(
                    "answer: %s" % (answer_text))

        features.append(
            InputFeatures(
                unique_id=unique_id,
                example_index=example_index,
                doc_span_index=doc_span_index,
                tokens=tokens,
                token_to_orig_map=token_to_orig_map,
                token_is_max_context=token_is_max_context,
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                cls_index=cls_index,
                p_mask=p_mask,
                paragraph_len=paragraph_len,
                start_position=start_position,
                end_position=end_position,
                is_impossible=span_is_impossible))
        unique_id += 1



